In [11]:
import codecs
import pickle
import numpy as np
import pandas as pd

In [12]:
#Reading the corpus

f = codecs.open("./output/kannada_tags.txt", "r", encoding="utf8")

raw = f.readlines()
#print(raw)

f.close()

In [13]:
#Splitting the words into kannada format

lines = []
l = []

for w in raw:
    w = w.encode('utf-8')
    x = w.strip(b'\n').split(b' ')
    if x[0] == b'*' or x[0] == b'.' or len(x) < 2:
        if len(l) > 0:
            lines.append(l)
        l = []
        continue
    l.append((x[0], x[1]))

In [14]:
sentences = []
labels = []
num_words = 0

for l in lines:
    s = []
    lab = []
    for w in l:
        x = w[0].decode('utf-8')
        y = w[1].decode('utf-8')
        s.append(x)
        num_words += 1
        #print(num_words)
        lab.append(y)
    labels.append(lab)
    sentences.append(s)
    print(sentences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
from gensim.models import Word2Vec

In [16]:
min_count = 1
size = 100
window = 20

model = Word2Vec(window=window, min_count=min_count, size=size)
print(model)
model.build_vocab(sentences)

Word2Vec(vocab=0, size=100, alpha=0.025)


In [17]:
#Train the model on the sentences
for i in range(1000):
    model.train(sentences, total_words=num_words, epochs=1)
    if i%100 == 0:
        print("Iteration ",i)

Iteration  0
Iteration  100
Iteration  200
Iteration  300
Iteration  400
Iteration  500
Iteration  600
Iteration  700
Iteration  800
Iteration  900


In [18]:
X = []
y = []
for idx,s in enumerate(sentences):
    x = []
    yy = []
    for wi,w in enumerate(s):
        if w in model:
            m = model[w]
            if wi > 1:
                m = np.hstack([m,model[s[wi-1]]])
            else:
                m = np.hstack([m,np.zeros_like(model[w])])
            
            if wi > 2:
                m = np.hstack([m,model[s[wi-2]]])
            else:
                m = np.hstack([m,np.zeros_like(model[w])])
            
            if wi > 3:
                m = np.hstack([m,model[s[wi-3]]])
            else:
                m = np.hstack([m,np.zeros_like(model[w])])
            
            if wi < len(s)-1:
                m = np.hstack([m,model[s[wi+1]]])
            else:
                m = np.hstack([m,np.zeros_like(model[w])])
            
            if wi < len(s)-2:
                m = np.hstack([m,model[s[wi+2]]])
            else:
                m = np.hstack([m,np.zeros_like(model[w])])
            
            if wi < len(s)-3:
                m = np.hstack([m,model[s[wi+3]]])
            else:
                m = np.hstack([m,np.zeros_like(model[w])])
            x.append(m)
    offset = 100 - len(x)
    pad_x = np.zeros_like(model[w]) - 1
    pad_x = np.hstack([pad_x,pad_x,pad_x,pad_x,pad_x, pad_x, pad_x])
    yy = yy + labels[idx]
    for i in range(offset):
        x.append(pad_x)
        yy.append("IRR")
        #print(idx," ",m)
        #print(m)
    X.append(x)
    y.append(yy)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [19]:
#Unique Tags
#Encoding the target labels
X = np.array(X)
y = np.array(y)
np.unique(y)

array(['CC', 'DEM', 'IRR', 'JJ', 'NEG', 'NN', 'NST', 'NUM', 'PRP', 'QF',
       'RB', 'RP', 'SYM', 'VM', 'WQ'], dtype='<U3')

In [20]:
original_shape = y.shape
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y.ravel()).reshape(original_shape)
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int64)

#### Saving the feature and label vectors

In [21]:
f = open("kannada-features.numpy", "wb")
np.save(f, X)
f.close()

In [22]:
f = open("kannada-labels.numpy", "wb")
np.save(f,y)
f.close()